In [3]:
# MAIN PACKAGES
import pandas as pd
import numpy as np
import sklearn
from tqdm import tqdm

# MODEL VALIDATION
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

#RFR SPECIFIC
from sklearn.ensemble import RandomForestRegressor


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This notebook trains + tests each encoding library to find the best perfoming encoding amino acid property matricies. These Matricies will be used to generate final predictions on witheld test set and novel mutant library.


Inputs:
1.   Combined Dataset Path (String)
2.   Desired Name of Dependent Variable (String)
3.   len(Sequence of Base Variant) (int)
4.   Property Matrix Path (String)

Outputs:
df_RFR
*   Dataframe (2 columns: Encoding Dataset Column + R Squared Achieved Column)
*   Saved as RFR_Encoding_Datasets.csv in the RFR/ Sub Folder


In [5]:
mutation_datset_path = '/content/drive/MyDrive/ml_paper_ipnys/backend_data/combined_dataset.csv'
len_sequence_of_base_variant = 451 # len(Sequence of Base Variant)
dependent_variable = 'Fluor Decay' # Desired Name of Dependent Variable
encoding_dataset_path = '/content/drive/MyDrive/ml_paper_ipnys/backend_data/full_property_matrix.csv'
parent_folder_path = '2023-07-22-ensemble-run/' #should be created in wrapper

In [6]:
data = pd.read_csv(mutation_datset_path)
position_cols = np.arange(0,len_sequence_of_base_variant)
position_cols = [str(i) for i in position_cols]
encoded_df = data[position_cols]

x_train, x_test, y_train, y_test = train_test_split(encoded_df,
                                                    data[dependent_variable],
                                                    test_size=0.20,
                                                    random_state=42)

In [7]:
# Read in encoding dataset
encoding_data = pd.read_csv(encoding_dataset_path, index_col = 0)

In [8]:
#initialize output dataframe
df_RFR = pd.DataFrame()

#Iterate through every encoding dataset + train model +
#record performance
for AA_property_dataset in tqdm(encoding_data.columns[1:],
                                desc = 'Property Datasets Encoded:'):

    # Make the train/test be on a copy to ensure there
    #is no data overwriting within loop
    x_train_copy = x_train.copy()
    x_test_copy = x_test.copy()

    #extract encoding data for specific iteration
    volume_dict = {'Amino Acid Code': encoding_data[encoding_data.columns[0]],
                   AA_property_dataset: encoding_data[AA_property_dataset]}
    volume_data = pd.DataFrame(volume_dict)


    #Some encoding datasets contain NaNs, I skip these
    #datasets since theyre incomplete
    #Next three lines check the encoding data for NaNs
    df = list(volume_data.iloc[:,1].values)
    T = np.isnan(df)
    TF = True in T

    #If there is no NaN, perform model training
    if TF == False:

        #initialize list to append to throughout training
        interlist = []

        #Use volume_data as a codex to translate sequence data...
        # amino acids will translate to float type data
        col_title = volume_data.columns[1]
        for row, sample in enumerate(volume_data['Amino Acid Code']):
            amino = sample
            replacement_value = float(volume_data[col_title].iloc[row])
            x_train_copy = x_train_copy.replace(amino,replacement_value)
            x_test_copy = x_test_copy.replace(amino,replacement_value)


        #Initialize Model and fit data to extract feature importances
        model = RandomForestRegressor()
        model.fit(x_train_copy,y_train)
        feat_importances = pd.Series(model.feature_importances_,
                                     index=x_train_copy.columns)


        ##Hyper Parameter Tuning
        #Grid Search approach, I test every iteration to find the best
        #combination of neighbors and features

        #Initialize the output lists to append to
        n_est = []
        test_r2s = []

        #For 21 possible best features
        for l in range(21):
            if l > 0:
                cols = list(feat_importances.nlargest(l).index)

                #extract l features from X_train/X_test
                x_train_copy_ = x_train_copy[cols]
                x_test_copy_ = x_test_copy[cols]

                #Initialize list of estimators to test
                n_estimators = [10, 15, 20, 25, 30, 35, 40,
                                45, 50, 55, 65, 75, 85, 100]

                #Initialize the output lists to append to
                n_mse_list = []
                n_r2_list = []

                for estimators in n_estimators:
                    #initialize model with n number of estimators
                    clf_RF = RandomForestRegressor(n_estimators=estimators,
                                                   random_state=42)
                    #Fit the Train data
                    clf_RF.fit(x_train_copy_, y_train)
                    #Predict the Test set and generate metrics of fit
                    y_RF = clf_RF.predict(x_test_copy_)
                    n_mse_list.append(mean_squared_error(y_test, y_RF))
                    n_r2_list.append(sklearn.metrics.r2_score(y_test, y_RF))

                #find which number of estimators led to the best R2 for
                # the test set
                best_est = n_estimators[n_mse_list.index(min(n_mse_list))]
                n_est.append(best_est)
                test_r2s.append(np.mean(n_r2_list))

        #find which iteration led to the greatest R2
        #est_best is the number of estimators when the
        #max R2 occured
        best_feats = test_r2s.index(max(test_r2s))
        est_best = n_est[best_feats]

        #similarly, extract the optimal number of features
        #by extracting the number of features that led to the
        #greatest R2
        cols = list(feat_importances.nlargest(best_feats + 1).index)
        x_train_copy_ = x_train_copy[cols]
        x_test_copy_ = x_test_copy[cols]

        #initialize new model + fit data
        clf = RandomForestRegressor(n_estimators = est_best,random_state = 42)
        clf.fit(x_train_copy_, y_train)

        #Create cross validation prediction
        y_pred = clf.predict(x_test_copy_)

        #Save the overall R2 for the tuned model
        r2 = sklearn.metrics.r2_score(y_test, y_pred)

        #append the data to a dataframe for export
        inter_df= pd.DataFrame({'Encoding Dataset': [AA_property_dataset[-11:]],
                                'Test Set R Squared' : [r2]})

        df_RFR = pd.concat([df_RFR,inter_df],ignore_index = True)

df_RFR = df_RFR.sort_values(by='Test Set R Squared', ascending = False)
df_RFR.to_csv(parent_folder_path + 'RFR/RFR_Encoding_Datasets.csv')


assert len(df_RFR) == 553
assert df_RFR['Test Set R Squared'].iloc[0] >= df_RFR['Test Set R Squared'].iloc[1]

Property Datasets Encoded:: 100%|██████████| 566/566 [5:42:03<00:00, 36.26s/it]


In [10]:
##FOR COLAB ONLY
df_RFR.to_csv('/content/drive/MyDrive/ml_paper_ipnys/RFR_Encoding_Datasets.csv')